In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, classification_report
import csv
from sklearn.model_selection import cross_validate

In [ ]:

def get_pixel_value(img, row, col):
  h, w = img.shape
  if row in range(h) and col in range(w):
    return img[row, col]
  else:
    return 0
  
def calculate_variance(numbers):
    epsilon = 1e-10
    if len(numbers) < 2:
        print("Variance requires at least two numbers")
        return epsilon
    mean = np.mean(numbers)
    squared_diff = [(x - mean) ** 2 for x in numbers]
    variance = sum(squared_diff) / (len(numbers) - 1)       
    return variance + epsilon;

def calculate_odlb(img_block):
    topLeft = img_block[0]
    topCenter = img_block[1]
    topRight = img_block[2]
    centerLeft = img_block[3]
    center = img_block[4]
    centerRight = img_block[5]
    buttomLeft = img_block[6]
    buttomCenter = img_block[7]
    buttomRight = img_block[8]

    firstPostiona1 = [topLeft - topCenter, topLeft-center, topLeft-topRight]
    secondPostiona1 = [buttomLeft - buttomCenter,buttomLeft - center, buttomLeft - centerLeft]
    thirdPositiona1 = [buttomRight - buttomCenter,buttomRight - center,buttomRight - centerRight]
    fourthPositiona1 = [topRight - topCenter,topRight - center,topRight - centerRight]
    firstPostiona2 = [topCenter - topRight,topCenter-center,topCenter-topLeft]
    secondPostiona2 = [centerLeft - buttomLeft,centerLeft - center, centerLeft - topLeft]
    thirdPositiona2 = [buttomCenter - buttomRight,buttomCenter - center,buttomCenter - buttomLeft]
    fourthPositiona2 = [centerRight - topRight,centerRight - center,centerRight - buttomRight]

    tl = np.sum(firstPostiona1)/calculate_variance(firstPostiona1)
    bl = np.sum(secondPostiona1)/calculate_variance(secondPostiona1)
    br = np.sum(thirdPositiona1)/calculate_variance(thirdPositiona1)
    tr = np.sum(fourthPositiona1)/calculate_variance(fourthPositiona1)
    ct = np.sum(firstPostiona2)/calculate_variance(firstPostiona2)
    cl = np.sum(secondPostiona2)/calculate_variance(secondPostiona2)
    cb = np.sum(thirdPositiona2)/calculate_variance(thirdPositiona2)
    cr = np.sum(fourthPositiona2)/calculate_variance(fourthPositiona2)

    t_tl = [1 if n >= tl else 0 for n in firstPostiona1]
    t_bl = [1 if n >= bl else 0 for n in secondPostiona1]
    t_br = [1 if n >= br else 0 for n in thirdPositiona1]
    t_tr = [1 if n >= tr else 0 for n in fourthPositiona1]
    t_ct = [1 if n >= ct else 0 for n in firstPostiona2]
    t_cl = [1 if n >= cl else 0 for n in secondPostiona2]
    t_cb = [1 if n >= cb else 0 for n in thirdPositiona2]
    t_cr = [1 if n >= cr else 0 for n in fourthPositiona2]

    con = np.concatenate((t_tl, t_bl, t_br, t_tr,t_ct,t_cl,t_cb,t_cr))

    split = (con[0:8],con[8:16],con[16:])

    decimals = []
    for bin_code in split:
        to_decimal = 0
        for i in range(len(bin_code)):
            bit = bin_code[i]
            to_decimal += bit * (2 ** i)
        decimals.append(to_decimal)

    return decimals

  
def ODLBP(image:str):
    """Compute the Multi Block local binary patterns (OD-LBP) of an image.

    OD-LBP, or Opponent Descriptor Local Binary Pattern, enhances traditional LBP techniques by incorporating color information through the use of opponent color spaces. This method processes each color channel separately, applying the LBP operator to capture detailed texture and color characteristics. By integrating color differences and intensity variations, ODLBP provides a more robust descriptor for image analysis tasks. It is particularly effective in environments where color attributes are significant, improving performance in applications such as object recognition and digital image processing.

    Parameters
    ----------
    image : (M, N) array
        2D grayscale image.
    
    Returns
    -------
    output : (D, M, N) array
        3 OD-LBP Image.
    """
    image = cv2.imread(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY).astype(np.int16)
    h, w = image.shape[:2]
    odlbp_image1 = np.zeros((h, w))
    odlbp_image2 = np.zeros((h, w))
    odlbp_image3 = np.zeros((h, w))
    for row in range(h):
        for col in range(w):
            img_block = [
                get_pixel_value(image, row-1, col-1),
                get_pixel_value(image, row-1, col),
                get_pixel_value(image, row-1, col+1),
                get_pixel_value(image, row, col-1),
                get_pixel_value(image, row, col),
                get_pixel_value(image, row, col+1),
                get_pixel_value(image, row+1, col-1),
                get_pixel_value(image, row+1, col),
                get_pixel_value(image, row+1, col+1),
            ]
            lbp_code = calculate_odlb(img_block)
            odlbp_image1[row, col] = lbp_code[0]
            odlbp_image2[row, col] = lbp_code[1]
            odlbp_image3[row, col] = lbp_code[2]
           
    return np.array([odlbp_image1, odlbp_image2, odlbp_image3])

def compute_odlbp_histogram(odlbp_images):
    """Compute the histogram for OD-LBP images based on the journal's methodology.

    Parameters
    ----------
    odlbp_images : tuple of (M, N) arrays
        3 OD-LBP images (OD-LBP1, OD-LBP2, OD-LBP3).

    Returns
    -------
    histogram : (6912,) array
        Concatenated histogram of all sub-regions and all 3 OD-LBP images.
    """
    odlbp_image1, odlbp_image2, odlbp_image3 = odlbp_images
    h, w = odlbp_image1.shape
    sub_region_size = h // 3  # Assuming 3x3 sub-regions
    histograms = []

    for odlbp_image in [odlbp_image1, odlbp_image2, odlbp_image3]:
        for i in range(3):
            for j in range(3):
                # Extract the sub-region
                row_start = i * sub_region_size
                row_end = (i + 1) * sub_region_size
                col_start = j * sub_region_size
                col_end = (j + 1) * sub_region_size

                # Compute histogram for the sub-region
                subregion = odlbp_image[row_start:row_end, col_start:col_end]
                hist, _ = np.histogram(subregion, bins=256, range=(0, 256))
                histograms.append(hist)

    # Concatenate all histograms and normalize
    final_histogram = np.concatenate(histograms)
    final_histogram = final_histogram.astype("float")
    return final_histogram

In [ ]:
def extract_odlbp_features(image_paths):
    features = []
    for image_path in image_paths:
        odlbp = ODLBP(image_path)
        hist = compute_odlbp_histogram(odlbp)
        features.append(hist)
    return np.array(features)

In [ ]:
def extract_odlbp_features_from_directory(directory_path,  output_file_name):
    categories = { "male" : 0, "female" : 1}

    image_paths = []
    labels = []

    for category, label in categories.items():
        category_path = os.path.join(directory_path, category)
        for img_name in sorted(os.listdir(category_path)):
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                img_path = os.path.join(category_path, img_name)    
                image_paths.append(img_path)
                labels.append(label)

    features_array = extract_odlbp_features(image_paths=image_paths)
    labels_array = np.array(labels)

    np.savez(f'{output_file_name}.npz', features=features_array, labels=labels_array)


In [ ]:
# scheme 1
extract_odlbp_features_from_directory(directory_path="../dataset/scheme-1", output_file_name="odlbp_1")
# scheme 1
extract_odlbp_features_from_directory(directory_path="../dataset/scheme-2", output_file_name="odlbp_2")